# ELEC 474 Prelab 
Matthieu Roux 20013052

In [1]:
import cv2
import numpy as np
import copy # used to create deep copies of the original pixel

In [2]:
img = cv2.imread("baboon.png")
x = y = red = green = blue = 0

# segment_pixels will hold all the points that are in the region
segment_pixels = {}
# next_pixel_stack holds all the points that could be added to the segment.
next_pixel_stack=[]

In [3]:
segment_color = [0, 255, 0] # should be a very bright green

## onMouse
The `onMouse` function handles the left click trigger on the image and obtains the coordinates of the click

In [4]:
def onMouse(event,_x,_y,flags,param):
    global x, y, green, blue, red, img, next_pixel_stack
    if event == cv2.EVENT_LBUTTONDOWN:
        green = img[_y,_x,0]
        blue = img[_y,_x,1]
        red = img[_y,_x,2]
        x = _x
        y = _y
        
        next_pixel_stack.append((_y,_x))
        segmentation()

In [5]:
def segmentation():
    global x, y, green, blue, red, img, next_pixel_stack, segment_pixels, segment_color
    og_pixel = copy.deepcopy(img[next_pixel_stack[0]])
    print(og_pixel)
    while len(next_pixel_stack) > 0:
        pixel = next_pixel_stack.pop()
        # check if the pixels are already in the degment
        if pixel in segment_pixels:
            continue
        if is_homogeneous_2(og_pixel, img[pixel]):
            segment_pixels[pixel] = True
            # update the pixel
            img[pixel] = segment_color
            # find the next potential pixels and add them to the stack
            next_pixels = get_next_pixels(pixel)
            next_pixel_stack.extend(next_pixels)

In [6]:
def get_diff(value1, value2):
    if value1 > value2:
        diff = value1 - value2
    else:
        diff = value2 - value1
    return diff
    
def is_homogeneous(og_pixel, pixel):
    global img, threshold
    diffs = [get_diff(og_pixel[i], pixel[i]) for i in range(3)]
    return diffs[0] <= threshold or diffs[1] <= threshold or diffs[2] <= threshold

def is_homogeneous_2(og_pixel, pixel):
    global img, threshold
    diffs = [get_diff(og_pixel[i], pixel[i]) for i in range(3)]
    homogeneous = diffs[0] <= threshold and diffs[1] <= threshold and diffs[2] <= threshold
    return homogeneous

In [7]:
def get_next_pixels(pixel):
    global img
    max_x = img.shape[1] - 1
    max_y = img.shape[0] - 1
    next_pixels = []
    if pixel[0] != 0:
        next_pixels.append((pixel[0] - 1, pixel[1]))
    if pixel[0] != max_y:
        next_pixels.append((pixel[0] + 1, pixel[1]))
    if pixel[1] != 0:
        next_pixels.append((pixel[0], pixel[1] - 1))
    if pixel[1] != max_x:
        next_pixels.append((pixel[0], pixel[1] + 1))
    return next_pixels

In [8]:
threshold = 20

def on_trackbar(val):
    global threshold
    threshold = val

In [ ]:
window_name = "image"
cv2.namedWindow(window_name)

cv2.createTrackbar("Threshhold", window_name, threshold , 100, on_trackbar)
keepGoing = True 
while(keepGoing):
    cv2.setMouseCallback(window_name, onMouse)
    # Wait a little bit for the image to re-draw
    key = cv2.waitKey(1)
    cv2.imshow(window_name, img)
    if key == ord("x"):
        keepGoing = False
    if key == ord("r"):
        img = cv2.imread("baboon.png")
        segment_pixels = {}

[ 63 189 222]
[ 45 185 221]
[ 13  91 241]
[ 74  89 235]
[177 161 198]


In [ ]:
cv2.destroyAllWindows()